In [20]:
import numpy as np

In [21]:
all_participants = np.load('../datasets/topicos_cc/s_t.npy')

In [22]:
all_participants.shape

(10, 8, 3, 4, 1600)

In [23]:
from scipy.signal import stft, resample, welch

all_participants_domains = []
for i in range(all_participants.shape[0]):
    participant = all_participants[i]
    step = 48
    segment = 64
    dataR = participant.reshape(24, 4, 1600)

    n_win = int((dataR.shape[-1] - segment) / step) + 1
    ids = np.arange(n_win) * step

    # Janelas do dado no dominio do tempo
    chunks_time = np.array([dataR[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

    # Janelas do dado no domínio da frequência
    _, _, chunks_freq = stft(dataR, fs=200, nperseg=128, noverlap=78)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    f, chunks_welch = welch(dataR, fs=200, nperseg=64, noverlap=32)

    domains = {
        'time': chunks_time,
        'freq': chunks_freq,
        'welch': chunks_welch,
        'f': f
    }

    all_participants_domains.append(domains)

print('Shape dos dados após a transformação:')
print(f'Dominio do tempo: {all_participants_domains[0]["time"].shape} - (Participantes, Classes+Ensaios, Canais, Janelas, Linhas)')
print(f'Dominio da frequência: {all_participants_domains[0]["freq"].shape} - (Participantes, Classes+Ensaios, Canais, Janelas, Linhas)')
print(f'Dominio da welch: {all_participants_domains[0]["welch"].shape} - (Participantes, Classes+Ensaios, Canais, Janelas)')

Shape dos dados após a transformação:
Dominio do tempo: (24, 4, 33, 64) - (Participantes, Classes+Ensaios, Canais, Janelas, Linhas)
Dominio da frequência: (24, 4, 33, 65) - (Participantes, Classes+Ensaios, Canais, Janelas, Linhas)
Dominio da welch: (24, 4, 33) - (Participantes, Classes+Ensaios, Canais, Janelas)


In [24]:
def psd(w):
    return np.abs(w) ** 2

def wamp(x, threshold):
    return np.sum(np.abs(np.diff(x)) > threshold, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def wl(x):
    return np.abs(np.diff(x, axis=-1)).sum(axis=-1)

def zc(x):
    return np.sum(np.diff(np.sign(x), axis=-1) != 0, axis=-1)

def fmd(w):
    return np.sum(psd(w), axis=-1) / 2

def fmn(w):
    _,_,_,M = w.shape
    f_j = (np.arange(1, M+1) * 200 )/ (2 * M)
    weighted_sum = np.sum(f_j * psd(w))
    mean_freq = weighted_sum / fmd(w)*2
    return mean_freq

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def mmnf(w):
    _,_,_,M = w.shape
    f_j = np.arange(1, M+1) * 200 / (2 * M) 
    weighted_sum = np.sum(f_j * np.abs(w), axis=-1)
    mean_freq = weighted_sum / mmdf(w)*2
    return mean_freq

In [25]:
all_participants_features = []
for i in range(len(all_participants_domains)):
    features = []
    participant = all_participants_domains[i]
    features.append(participant['welch'])
    features.append(var(participant['time']))
    features.append(rms(participant['time']))
    features.append(wamp(participant['time'], 0))
    features.append(wl(participant['time']))
    features.append(zc(participant['time']))
    features.append(fmd(participant['freq']))
    features.append(mmdf(participant['freq']))
    features.append(fmn(participant['freq']))
    features.append(mmnf(participant['freq']))

    all_participants_features.append(np.array(features))

print(len(all_participants_features))
print(all_participants_features[0].shape)
all_participants_features = np.array(all_participants_features)
print(f'Shape dos dados após a extração de características: {all_participants_features.shape}')

all_participants_features = all_participants_features.swapaxes(2, 3)
all_participants_features = all_participants_features.reshape(10, 40, 24 * 33)
all_participants_features = all_participants_features.swapaxes(1, 2)

print(f'Shape dos dados após a extração de características: {all_participants_features.shape}')

np.save('../datasets/topicos_cc/s_t_features.npy', all_participants_features)

10
(10, 24, 4, 33)
Shape dos dados após a extração de características: (10, 10, 24, 4, 33)
Shape dos dados após a extração de características: (10, 792, 40)
